In [14]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# 연결 변수
AWS_ACCESS_KEY_ID =""
AWS_SECRET_ACCESS_KEY = ""
AWS_DEFAULT_REGION = "ap-northeast-2"

# spark session 생성시 aws와 연동하기
def aws_connect_spark(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY):
    # 설정
    conf = (
        SparkConf()
        .setAppName("MY_APP") # replace with your desired name
        .set("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0,org.apache.hadoop:hadoop-aws:3.3.2")
        .set("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID)
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY)
        .set("spark.sql.shuffle.partitions", "4") # default is 200 partitions which is too many for local
        .setMaster("local[*]") # replace the * with your desired number of cores. * for use all.
    )

    # spark 생성
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    
    return spark

# S3와 연결된 Spark로부터 해당 s3 table path로부터 Delta Table 가져오기
def s3_read_deltatable(spark, delta_table_path, version):
    df = spark.read.format("delta").option("versionAsOf", version).load(delta_table_path)
    
    return df

In [15]:
delta_table_path = "s3a://donghee-deltalake-test/delta-table-quickstart"


In [21]:
spark = aws_connect_spark(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
df0 = s3_read_deltatable(spark, delta_table_path, 0) # Version 0 = Raw Data
df1 = s3_read_deltatable(spark, delta_table_path, 1) # Version 1
df2 = s3_read_deltatable(spark, delta_table_path, 2) # Version 2
df0.show()
df1.show()
df2.show()

+---+
| id|
+---+
|  2|
|  4|
|  0|
|  3|
|  1|
+---+

+---+
| id|
+---+
| 37|
| 35|
| 39|
| 33|
| 36|
| 38|
| 31|
| 32|
| 34|
| 30|
+---+

+---+
| id|
+---+
| 37|
| 35|
| 39|
| 33|
|138|
|136|
|130|
|132|
| 31|
|134|
+---+

